In [4]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
import numpy as np
import gc
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

tqdm.pandas()

gm = gpd.read_parquet('data/gemeenten_concatenated.parquet')

categorisation = pd.read_excel('data/categorisation.xlsx')
L0_cats = categorisation['L0 category'].unique()
L1_cats = categorisation['L1 category'].unique()

In [5]:
gm.head(3)

,gemeentecode,gemeentenaam,H2O,OAD,STED,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,P_00_14_JR,...,RE_L0_1,RE_L1_1,RE_L0_2,RE_L1_2,L0_altieri_0,L1_altieri_0,L0_altieri_1,L1_altieri_1,L0_altieri_2,L1_altieri_2
0,GM0014,Groningen,NEE,3427.0,1.0,1284.0,238147.0,118198.0,119949.0,12.0,...,0.083836,0.092403,0.081681,0.088819,5.404861,6.998096,6.066780,8.816592,6.047942,8.296879
1,GM0034,Almere,NEE,1629.0,2.0,1725.0,222825.0,110589.0,112236.0,18.0,...,0.227224,0.241199,0.215799,0.236978,4.866573,5.913394,5.859722,8.110194,5.807034,7.651038
2,GM0037,Stadskanaal,NEE,840.0,4.0,273.0,32135.0,15844.0,16291.0,14.0,...,0.030424,0.053134,0.033571,0.052019,5.748026,6.916790,4.928690,6.304145,4.768895,5.847494


In [7]:
for idx, row in tqdm(gm.iterrows(), total=gm.shape[0]):
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{row['gemeentenaam']}.parquet")
    # amenities = amenities[amenities.geometry.within(row.geometry)]
    for filter_i in [0, 1, 2]:
        
        # get and apply filter
        L0_filter, L1_filter = ec.getfilter(filter_i)
        
        amenities_f = amenities[~amenities.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_f = amenities_f[
                    ~( (amenities_f.L0_category == key) & (amenities_f.L1_category.isin(value)) )
                ]
        
        for cat in L0_cats:
            gm.loc[idx, f'L1_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L0_category == cat])
        
        for cat in L1_cats:
            gm.loc[idx, f'L0_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L1_category == cat])

    del amenities
    del amenities_f
    gc.collect()

gm.to_parquet('data/gemeenten_concatenated_counted.parquet') 

  9%|▉         | 30/342 [00:16<02:55,  1.78it/s]


,gemeentecode,gemeentenaam,H2O,OAD,STED,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,P_00_14_JR,...,L0_2_count_Bars and cafes,L0_2_count_Stands and takeaway,"L0_2_count_Bus, tram and metro",L0_2_count_Public transport,L0_2_count_Cycling,L0_2_count_Services,L0_2_count_Fuel and charge,L0_2_count_Parking,L0_2_count_Train,L0_2_count_Waste management
0,GM0014,Groningen,NEE,3427.0,1.0,1284.0,238147.0,118198.0,119949.0,12.0,...,248.0,6.0,656.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GM0034,Almere,NEE,1629.0,2.0,1725.0,222825.0,110589.0,112236.0,18.0,...,44.0,2.0,517.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,GM0037,Stadskanaal,NEE,840.0,4.0,273.0,32135.0,15844.0,16291.0,14.0,...,7.0,0.0,155.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,GM0047,Veendam,NEE,1013.0,3.0,364.0,27616.0,13863.0,13753.0,14.0,...,12.0,1.0,134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,GM0050,Zeewolde,NEE,842.0,4.0,96.0,23692.0,12003.0,11689.0,17.0,...,7.0,1.0,136.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
5,GM0059,Achtkarspelen,NEE,437.0,5.0,275.0,28149.0,14352.0,13797.0,17.0,...,14.0,0.0,114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,GM0060,Ameland,NEE,266.0,5.0,68.0,3840.0,1938.0,1902.0,14.0,...,15.0,1.0,127.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
7,GM0072,Harlingen,NEE,1092.0,3.0,649.0,16188.0,8115.0,8073.0,14.0,...,13.0,1.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,GM0074,Heerenveen,NEE,1109.0,3.0,272.0,51637.0,25706.0,25931.0,15.0,...,35.0,0.0,208.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
9,GM0080,Leeuwarden,NEE,2196.0,2.0,535.0,127073.0,63248.0,63825.0,15.0,...,95.0,6.0,424.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
